In [ ]:
import glob
import os
import random
import socket
import timeit
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn.functional as F
from torchvision import transforms
from sklearn.metrics import (auc, confusion_matrix,
                             precision_recall_fscore_support, precision_score,
                             recall_score, roc_curve)

from sklearn.metrics.pairwise import cosine_similarity
from tensorboardX import SummaryWriter
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from PIL import Image

from model import Con2DAutoencoder
from dataloaders.dataset_ZSL import VideoDataset, ImageDataset
from network import Pac3D_ZSL_model


In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) 
    os.environ['PYTHONHASHSEED'] = str(seed)

In [ ]:
############################
####    Parameters      ####
############################
nEpochs = 10  # Number of epochs for training
resume_epoch = 0  # Default is 0, change if want to resume
useTest = True # See evolution of the test set when training
nTestInterval = 10 # Run on test set every nTestInterval epochs
save_epoch = 10 # Store a model every save_epoch
lr = 1e-3 # Learning rate
clip_len = 256 # frames of each video

###################################
####    Options of Dataset     ####
###################################
dataset = 'CIC-IDS2018-ZSL-DoS' 
domain = 'DoS' # DoS, DDoS, Auth, Web, Other    
saveName = 'Pac3D' + '-' + dataset

if dataset == 'CIC-IDS2018':
    set_seed(7)
elif dataset == 'CIC-IDS2018-ZSL-DDoS':
    set_seed(35)
elif dataset == 'CIC-IDS2018-ZSL-DoS':
    set_seed(6)
elif dataset == 'CIC-IDS2018-ZSL-Auth':
    set_seed(9)
elif dataset == 'CIC-IDS2018-ZSL-Web':
    set_seed(2)
elif dataset == 'CIC-IDS2018-ZSL-Other':
    set_seed(21)
else:
    print('No Dataset')
    raise NotImplementedError

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device being used:", device)

In [ ]:
embedding_path = '/SSD/p76111262/label_embedding_32'
vector_map = []
seen_vector_map = []
unseen_vector_map = []

if domain == 'DoS':
    attack_list = ['DoS_SlowHTTPTest', 'DoS_Slowloris', 'DoS_Hulk', 'DoS_GoldenEye']
    unseen_class = ['DoS_Slowloris']
    seen_class = ['DoS_SlowHTTPTest', 'DoS_Hulk', 'DoS_GoldenEye']
elif domain == 'DDoS':
    attack_list = ['DDoS_LOIC-HTTP', 'DDoS_HOIC', 'DDoS_LOIC-UDP'] 
    unseen_class = ['DDoS_LOIC-UDP']
    seen_class = ['DDoS_LOIC-HTTP', 'DDoS_HOIC']
elif domain == 'Auth':
    attack_list = ['BruteForce-SSH', 'BruteForce-FTP']
    unseen_class = ['BruteForce-FTP']
    seen_class = ['BruteForce-SSH']
elif domain == 'Web':
    attack_list = ['BruteForce-XSS', 'BruteForce-Web', 'SQL-Injection']
    unseen_class = ['SQL-Injection']
    seen_class = ['BruteForce-XSS', 'BruteForce-Web']
elif domain == 'Other':
    attack_list = ['Infiltration', 'Botnet']
    unseen_class = ['Infiltration']
    seen_class = ['Botnet']

print("Domain:", domain)
print("Attack List:", attack_list)
print("Seen Class:", seen_class)
print("Unseen Class:", unseen_class)

for a in attack_list:
    file_name = os.path.join(embedding_path, f'{a}.npy')
    vector_map.append(np.load(file_name))
for seen in seen_class:
    file_name = os.path.join(embedding_path, f'{seen}.npy')
    seen_vector_map.append(np.load(file_name))
for unseen in unseen_class:
    file_name = os.path.join(embedding_path, f'{unseen}.npy')
    unseen_vector_map.append(np.load(file_name))

vector_map_tensors = [torch.tensor(vector, dtype=torch.float32) for vector in vector_map]
seen_vector_map_tensors = [torch.tensor(vector, dtype=torch.float32) for vector in seen_vector_map]
unseen_vector_map_tensors = [torch.tensor(vector, dtype=torch.float32) for vector in unseen_vector_map]
vector_map_tensor = torch.stack(vector_map_tensors)
seen_vector_map_tensor = torch.stack(seen_vector_map_tensors)
unseen_vector_map_tensor = torch.stack(unseen_vector_map_tensors)

In [ ]:

##########################################
####   Set Model result saving dir    ####
##########################################
save_dir_root = os.path.join("/SSD/p76111262/")
resume_epoch = 0

if resume_epoch != 0:
    runs = sorted(glob.glob(os.path.join(save_dir_root, 'run', 'run_*')))
    run_id = int(runs[-1].split('_')[-1]) if runs else 0
else:
    runs = sorted(glob.glob(os.path.join(save_dir_root, 'run', 'run_*')))
    if len(runs) == 0:
        run_id = 0
    else:
        run_id = max([int(i.split('_')[-1]) for i in runs]) + 1
save_dir = os.path.join(save_dir_root, 'run', 'run_' + str(run_id))


In [ ]:
# Function to find the optimal threshold from ROC curve
def find_optimal_threshold(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]
    return optimal_threshold

# Define the test function with accuracy calculation
def test(model, dataloader):
    model.eval()
    total_loss = 0
    outputs = []
    labels_list = []

    with torch.no_grad():
        for data, labels in dataloader:
            images = data.to(device)
            reconstructed_images = model(images)
            loss = criterion(reconstructed_images, images)
            total_loss += loss.item()

            reconstruction_error = torch.mean((reconstructed_images - images) ** 2, dim=[1, 2, 3]).cpu().numpy()
            outputs.extend(reconstruction_error)
            labels_list.extend(labels.cpu().numpy())

    average_loss = total_loss / len(dataloader)
    outputs = np.array(outputs)
    labels_list = np.array(labels_list)
    optimal_threshold = find_optimal_threshold(labels_list, outputs)
    # 四捨五入到小數點第5位
    optimal_threshold = round(optimal_threshold, 5)

    predicted_labels = (outputs > optimal_threshold).astype(int)
    accuracy = np.mean(predicted_labels == labels_list)
    print(f'Test Loss: {average_loss:.4f}, Accuracy: {accuracy:.4f}, Optimal Threshold: {optimal_threshold:.4f}')
    return optimal_threshold

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# Load dataset
AE_train_dataset = ImageDataset(root_dir='/SSD/p76111262/CIC-IDS2018-ZSL/DoS/train', transform=transform, unseen_class=unseen_class)
AE_test_dataset = ImageDataset(root_dir='/SSD/p76111262/CIC-IDS2018-ZSL/DoS/test', transform=transform, unseen_class=unseen_class)
print(f'Number of train images: {len(AE_train_dataset)}')
print(f'Number of test images: {len(AE_test_dataset)}')
AE_train_dataloader = DataLoader(AE_train_dataset, batch_size=10, shuffle=True)
AE_test_dataloader = DataLoader(AE_test_dataset, batch_size=4, shuffle=False)

# Model
AE_model = Con2DAutoencoder().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(AE_model.parameters(), lr=1e-3)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for data, _ in AE_train_dataloader:
        img = data.to(device)
        # Forward pass
        output = AE_model(img)
        loss = criterion(output, img)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

optimal_threshold = test(AE_model, AE_test_dataloader)

In [ ]:
model = Pac3D_ZSL_model.Pac3DClassifier(layer_sizes=(2, 2, 2, 2))
train_params = model.parameters()

In [ ]:
######################################
####   Load model & parameters    ####
######################################
criterion = nn.CosineEmbeddingLoss()
optimizer = optim.Adam(train_params, lr=lr, weight_decay=5e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)  # the scheduler divides the lr by 10 every 5 epochs

if resume_epoch == 0:
    print("Training {} from scratch...".format('Pac3D'))
else:
    checkpoint = torch.load(os.path.join(save_dir, 'models', saveName + '_epoch-' + str(resume_epoch - 1) + '.pth.tar'),
                    map_location=lambda storage, loc: storage)   # Load all tensors onto the CPU
    print("Initializing weights from: {}...".format(
        os.path.join(save_dir, 'models', saveName + '_epoch-' + str(resume_epoch - 1) + '.pth.tar')))
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['opt_dict'])

print('Total params: %.2fM' % (sum(p.numel() for p in model.parameters()) / 1000000.0))

model.to(device)
criterion.to(device)

log_dir = os.path.join(save_dir, 'models', datetime.now().strftime('%b%d_%H-%M-%S') + '_' + socket.gethostname())
print("log dir:", log_dir)
writer = SummaryWriter(log_dir=log_dir)


In [ ]:

########################
####   Load Data    ####
########################
print('Training model on {} dataset...'.format(dataset))
train_dataloader = DataLoader(VideoDataset(dataset=dataset, split='train', clip_len=clip_len, embedding_map=vector_map, attack_list=seen_class), batch_size=4, shuffle=True, num_workers=0)
test_dataloader  = DataLoader(VideoDataset(dataset=dataset, split='test', clip_len=clip_len, embedding_map=vector_map, attack_list=attack_list), batch_size=4, shuffle=False, num_workers=0)

train_size = len(train_dataloader.dataset)
test_size = len(test_dataloader.dataset)


In [ ]:
target = torch.ones(train_size, dtype=torch.float32, device=device)
train_losses = []
train_accs = []
y_pred = []
y_true = []

for epoch in range(resume_epoch, nEpochs):
    start_time = timeit.default_timer()
    # reset the running loss and corrects
    running_loss = 0.0
    running_corrects = 0.0

    # set model to train mode
    model.train()

    for inputs, embedding, label in tqdm(train_dataloader):
        # move inputs and labels to the device the training is taking place on
        inputs = Variable(inputs, requires_grad=True).to(device)
        embedding = Variable(embedding).to(device)
        label = Variable(label).to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        batch_size = outputs.size(0)
        target = torch.ones(batch_size, device=outputs.device)
        loss = criterion(outputs, embedding, target)

        loss.backward()
        optimizer.step()
        scheduler.step()

        running_loss += loss.item() * inputs.size(0)
        seen_vector_map_tensor = seen_vector_map_tensor.to(device)  
        similarities = F.cosine_similarity(outputs.unsqueeze(1), seen_vector_map_tensor.unsqueeze(0), dim=2)
        preds = torch.argmax(similarities, dim=1)
        running_corrects += torch.sum(preds == label)
        
    epoch_loss = running_loss / train_size
    epoch_acc = running_corrects.double() / train_size

    writer.add_scalar('data/train_loss_epoch', epoch_loss, epoch)
    writer.add_scalar('data/train_acc_epoch', epoch_acc, epoch)
    train_losses.append(epoch_loss)
    train_accs.append(epoch_acc)

    print("[train] Epoch: {}/{} Loss: {} Acc: {}".format(epoch+1, nEpochs, epoch_loss, epoch_acc))
    stop_time = timeit.default_timer()
    print("Execution time: " + str(stop_time - start_time) + "\n")

    if epoch % save_epoch == (save_epoch - 1):
        torch.save({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'opt_dict': optimizer.state_dict(),
        }, os.path.join(save_dir, 'models', saveName + '_epoch-' + str(epoch) + '.pth.tar'))
        print("Save model at {}\n".format(os.path.join(save_dir, 'models', saveName + '_epoch-' + str(epoch) + '.pth.tar')))

writer.close()
torch.save(model.state_dict(), "/SSD/p76111262/"+'Pac3D_run'+str(run_id)+".pth")


In [ ]:
def test_model_with_cosine_similarity(model, test_dataloader, device, optimal_threshold):
    model.eval()
    # AE_model.eval()
    running_corrects = 0

    y_pred = []
    y_true = []

    for (inputs, embedding, label), (AE_input, AE_labels) in tqdm(zip(test_dataloader, AE_test_dataloader), total=len(test_dataloader)):
        inputs = inputs.to(device)
        label = label.to(device)

        images = AE_input.to(device)
        with torch.no_grad():
            reconstructed_images = AE_model(images)
        reconstruction_error = torch.mean((reconstructed_images - images) ** 2, dim=[1, 2, 3]).cpu().numpy()
        
        is_seen = reconstruction_error < optimal_threshold
        for i in range(inputs.size(0)):
            if is_seen[i]:
                vector_map_tensor = seen_vector_map_tensor.to(device)
            else:
                vector_map_tensor = unseen_vector_map_tensor.to(device)

        with torch.no_grad():
            outputs = model(inputs[i].unsqueeze(0))

        # 計算每個輸出與所有標籤向量之間的 cosine similarity
        similarities = F.cosine_similarity(outputs, vector_map_tensor.unsqueeze(0), dim=1)
        pred = similarities.argmax().item()  # 預測為最相似向量的索引
        correct = (pred == label).sum().item()
        running_corrects += correct

        y_pred.append(pred)
        y_true.append(label[i].item())

    epoch_acc = running_corrects / len(test_dataloader.dataset)
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')

    print("[Test] Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}".format(epoch_acc, precision, recall))
    return y_true, y_pred

# 调用测试函数
y_true, y_pred = test_model_with_cosine_similarity(model, test_dataloader, device, optimal_threshold)

In [ ]:
print("y_pred:", y_pred)
print("y_true:", y_true)
precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", f1)

In [ ]:
import matplotlib.pyplot as plt
x = list(range(nEpochs))
plt.plot(x, train_losses)
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.savefig(save_dir + '/training_loss.png')
plt.show()

In [ ]:

# 製作混淆矩陣
cf_matrix = confusion_matrix(y_true, y_pred)                               
# 計算每個class的accuracy
per_cls_acc = cf_matrix.diagonal()/cf_matrix.sum(axis=0)                   

class_names = []
label_txt = os.path.join('dataloaders', dataset + ".txt")  # 這裡要改成你的label.txt路徑
with open(label_txt, 'r') as f:
    for line in f:
        class_names.append(line.strip())
        
print(class_names)
print(per_cls_acc)                                            

# 開始繪製混淆矩陣並存檔
df_cm = pd.DataFrame(cf_matrix, class_names, class_names)    
plt.figure(figsize = (9,6))
sns.heatmap(df_cm, annot=True, fmt="d", cmap='BuGn')
plt.xlabel("prediction")
plt.ylabel("label (ground truth)")
plt.savefig(save_dir + '/confusion_matrix.png')